In [ ]:
# ch09 01.ipynb

### 딥러닝 기반 감성분석

In [ ]:
## 딥러닝모델에 넣기위한 텍스트 전처리 방법 정리
# 데이터 - 토큰화 & 정수 인코딩 - 시퀀스 패딩
# baseline 일반 신경망 Dense
# Simple RNN 기울기소실문제
# Bidirectional LSTM 양방향 학습

In [ ]:
# Tokenization 텍스트를 숫자로 변환
# Word Embedding 단어를 고차원 벡터로 변환
# Sequence Padding 길이가 다른 문장을 같은 크기로 고정
# Simple RNN
# LSTM            RNN 의 경사소실 문제 해결 - 장기기억 소실
# Bidirrctional LSTM 양방향 콘텐츠 학습

In [ ]:
# 토큰화 : 수치로 변환하는 과정
# 토크나이저 3단계
    # 1. fit_on_text 가장 빈도가 높은 단어의 인덱스를 구축해서 딕셔너리
    # 2. texts_to_sequence 각 문서를 점수 시퀀스 변환
    # 3. pd_sequence 길이 정규화(같은 길이)

In [ ]:
# 딥러닝에서 워드 임베딩 레이어 :
# 단어를 모델이 계산할 수 있는 숫자 형태(숫자 벡터)로 바꾸고,
# 벡터의 길이를 일정하게 맞춘 것

In [3]:
sample_reviews = [
    "this movie is great and wonderful",
    "bad movie with poor acting",
    "great movie absolutely wonderful"
]

In [1]:
# 파이토치 버젼
from collections import Counter
import numpy as np
import torch

In [5]:
# 1. 단어 분할 및 빈도 계산
all_words = []
for i in [review.split() for review in sample_reviews]:
    all_words.extend(i)

In [ ]:
# 단어 빈도
word_freq = Counter(all_words)
word_freq.most_common(2) # 빈도수가 높은 단어 상위 2개 조회

[('movie', 3), ('great', 2)]

In [12]:
# 2. Tokenizer 구현
class SimpleTokenizer:
    def __init__(self, num_words = 10, oov_token = 'UNK'):
        self.num_words = num_words   # 상위 몇 개 단어까지 사용할지
        self.oov_token = oov_token   # 사전에 없는 단어를 위한 토큰
        self.word_index = {oov_token: 1}  # OOV 토큰은 1번 인덱스로 시작
        self.index_word = {1: oov_token}  # 인덱스로 단어도 조회 가능

    def fit_on_texts(self, texts):
        '''
        문장을 단어 인덱스로 변환
        '''
        all_words = []
        for i in [review.split() for review in sample_reviews]:
            all_words.extend(i)
        word_freq = Counter(all_words)
        # 빈도 높은 순서로 인덱스 부여
        # oov 토큰을 1로 설정
        self.word_index[self.oov_token] =1
        self.index_word[1]  = self.oov_token
        idx = 2
        for word, _ in word_freq.most_common(self.num_words-1):
            self.word_index[word] = idx
            self.index_word[idx] = word
            idx += 1

    def texts_to_sequences(self, texts):
        '''
        텍스트를 정수 시퀀스로 변환
        '''
        sequences = []
        for text in texts:
            seq = []
            for word in text.split():
                # 단어가 vocabulart에 있으면 인덱스를 사용, 없으면 oov
                word_index = self.word_index.get(word, 1)
                seq.append(word_index)
            sequences.append(seq)
        return sequences
    
# Tokenizer 생성 및 학습
tokenizer = SimpleTokenizer(num_words=10, oov_token='UUK')
tokenizer.fit_on_texts(sample_reviews)
# tokenizer.word_index

In [ ]:
# 텍스트를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(sample_reviews)
sequences

In [ ]:
sample_reviews

In [ ]:
# 패딩 구현 - 문자열의 길이를 동일하게 맞춘다